In [1]:
import pycuda.driver as drv
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [2]:
import numpy as np

In [3]:
n = 100

X = np.random.rand(n)
Y = np.random.rand(n)
Ed = np.random.rand(n)
EventID = np.sort(np.random.randint(0, 3, size=n)) 

In [120]:
import uproot
import utils
import glob
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
filenames = glob.glob('../cache/*.root')
def decode_energy(fn):
    return utils.hex_to_float(fn.split('_')[-1].replace('.root', '').replace('o', '.'))
fn = filenames[0]
energy = decode_energy(fn)

In [4]:
with uproot.open(fn) as file:
    rawEventID = file['LXe;1']['Event'].array(library='np')
    rawEd = -file['LXe;1']['Ed'].array(library='np')
    rawX = file['LXe;1']['X'].array(library='np')
    rawY = file['LXe;1']['Y'].array(library='np')
    rawZ = file['LXe;1']['Z'].array(library='np')

In [121]:
mask = rawEd > 0
mask &= (rawEventID >= 0) & (rawEventID < 50)

Ed = rawEd[mask]
X = rawX[mask]
Y = rawY[mask]
Z = rawZ[mask]
EventID = rawEventID[mask]

In [ ]:
def make_gpu_args(Ed, X, Y, EventID):
    # in
    num_event = np.asarray(len(np.unique(EventID))).astype(np.int32)

    # in
    params = np.asarray([6., 15.0]).astype(np.float32)
    
    # in
    X = np.asarray(X).astype(np.float32)
    Y = np.asarray(Y).astype(np.float32)
    Ed = np.asarray(Ed).astype(np.float32)

    # in
    event_end = np.argwhere(np.diff(EventID, prepend=-1, append=num_event+1)!=0).ravel()
    event_end = np.asarray(event_end).astype(np.int32)
    
    # placeholders
    # in
    X_edge = np.zeros(2 * len(X)).astype(np.float32)
    Y_edge = np.zeros(2 * len(Y)).astype(np.float32)

    # in
    X_edge_ID = np.zeros(2 * len(X)).astype(np.int32)
    Y_edge_ID = np.zeros(2 * len(Y)).astype(np.int32)

    # in
    _, num_hits = np.unique(EventID, return_counts=True)
    weight = np.zeros(np.sum((2*num_hits-1)**2)).astype(np.float32)

    # in
    recomb = np.zeros(num_event).astype(np.float32)

    # in
    fcache = np.zeros(4 * len(X)).astype(np.float32)
    icache = np.zeros(4 * len(X)).astype(np.int32)
    
    mem_bytes = np.sum([arr.nbytes for arr in [num_event, params, X, Y, Ed, event_end, X_edge, Y_edge, X_edge_ID, Y_edge_ID, weight, recomb, fcache, icache]])
    print("Memory used %i MB" % (mem_bytes/1e6))
    
    return (
        drv.In(X),
        drv.In(Y),
        drv.In(Ed),
        drv.In(event_end),
        drv.In(num_event),
        drv.In(params),
        drv.In(X_edge),
        drv.In(Y_edge),
        drv.In(X_edge_ID),
        drv.In(Y_edge_ID),
        drv.InOut(weight),
        drv.InOut(recomb),
        drv.In(fcache),
        drv.In(icache)
    )

In [102]:
# in
num_event = np.asarray(len(np.unique(EventID))).astype(np.int32)

# in
params = np.asarray([6., 15.0]).astype(np.float32)

In [103]:
# in
X = np.asarray(X).astype(np.float32)
Y = np.asarray(Y).astype(np.float32)
Ed = np.asarray(Ed).astype(np.float32)

# in
event_end = np.argwhere(np.diff(EventID, prepend=-1, append=num_event+1)!=0).ravel()
event_end = np.asarray(event_end).astype(np.int32)

In [104]:
# placeholders

# in
X_edge = np.zeros(2 * len(X)).astype(np.float32)
Y_edge = np.zeros(2 * len(Y)).astype(np.float32)

# in
X_edge_ID = np.zeros(2 * len(X)).astype(np.int32)
Y_edge_ID = np.zeros(2 * len(Y)).astype(np.int32)

# in
_, num_hits = np.unique(EventID, return_counts=True)
weight = np.zeros(np.sum((2*num_hits-1)**2)).astype(np.float32)

# in
recomb = np.zeros(num_event).astype(np.float32)

# in
fcache = np.zeros(4 * len(X)).astype(np.float32)
icache = np.zeros(4 * len(X)).astype(np.int32)

In [105]:
weight.nbytes/1e6

300.206856

In [106]:
gpu_kernels = open('./CudaKernels.cpp', 'r').read()
mod = SourceModule(gpu_kernels)
func = mod.get_function('calculate_recomb')

In [107]:
func(
    drv.In(X),
    drv.In(Y),
    drv.In(Ed),
    drv.In(event_end),
    drv.In(num_event),
    drv.In(params),
    drv.In(X_edge),
    drv.In(Y_edge),
    drv.In(X_edge_ID),
    drv.In(Y_edge_ID),
    drv.InOut(weight),
    drv.InOut(recomb),
    drv.In(fcache),
    drv.In(icache),
    block=(100,1,1)
)

In [108]:
np.abs(recomb - np.array([np.sum(Ed[event_end[i]:event_end[i+1]]) for i in np.arange(len(event_end)-1)]))#[32]

array([1.9073486e-05, 1.1444092e-05, 3.8146973e-06, 1.3351440e-05,
       1.7166138e-05, 2.2888184e-05, 3.8146973e-06, 1.5258789e-05,
       3.8146973e-06, 3.8146973e-06, 1.7166138e-05, 1.1444092e-05,
       7.6293945e-06, 7.6293945e-06, 5.7220459e-06, 1.1444092e-05,
       1.3351440e-05, 9.5367432e-06, 5.7220459e-06, 9.5367432e-06,
       7.6293945e-06, 1.3351440e-05, 1.7166138e-05, 9.5367432e-06,
       7.6293945e-06, 2.6702881e-05, 0.0000000e+00, 5.7220459e-06,
       5.7220459e-06, 0.0000000e+00, 1.9073486e-06, 1.3351440e-05,
       2.6620144e+01, 3.8146973e-06, 1.3351440e-05, 7.6293945e-06,
       1.9073486e-05, 1.7166138e-05, 1.9073486e-06, 1.1444092e-05,
       1.7166138e-05, 0.0000000e+00, 1.9073486e-05, 1.9073486e-06,
       1.5258789e-05, 5.7220459e-06, 2.4795532e-05, 5.7220459e-06,
       1.9073486e-05, 9.5367432e-06], dtype=float32)

In [122]:
recomb

array([27.952724 , 33.34397  , 28.236713 , 29.024992 , 30.386484 ,
       29.032461 , 29.739925 , 30.790277 , 28.040995 , 28.432306 ,
       31.648596 , 33.22568  , 28.098562 , 32.221962 , 28.05479  ,
       29.412214 , 28.802011 , 29.098103 , 30.436262 , 28.002205 ,
       28.8129   , 28.708092 , 29.171549 , 28.681875 , 27.949827 ,
       33.311165 , 28.322334 , 28.415289 , 28.543217 , 28.131073 ,
       28.727602 , 29.75525  ,  1.4749134, 29.007195 , 28.072088 ,
       28.567566 , 30.231838 , 29.568323 , 30.469774 , 30.746002 ,
       28.794193 , 28.016611 , 27.94323  , 27.945541 , 28.804346 ,
       28.335073 , 29.044119 , 28.179352 , 30.553263 , 28.388826 ],
      dtype=float32)

In [123]:
energy

15.0

In [124]:
e = np.array([-0.0127987, -0.0263011, -0.00554445, -0.000230696, -0.0751037, -0.0111075, -0.0307128, -0.0285462, -0.00197146, -0.0164271, -0.0497265, -0.00569493, -0.0375349, -0.0134435, -0.0198317, -0.011446, -0.0178754, -0.00542379, -0.0137995, -0.0318287, -0.0178632, -0.00119826, -0.0771457, -0.015979, -0.168465, -0.0158306, -0.0432541, -0.0191583, -0.0122571, -0.0121755, -0.0314885, -0.0261864, -0.021141, -1.11217e-05, -0.000356849, -0.000428301, -0.0483484, -0.0390908, -0.0278257, -0.000347758, -0.0583748, -0.00031675, -0.00202936, -0.012787, -0.0190901, -0.0416937, -0.000185156, -0.0123094, -0.189102, -0.0126123, -0.15905, -0.0218388, -0.0741539, -0.0269603, -0.0272275, -0.00598487, -0.0115531, -0.014322, -0.0185023, -0.0141965, -0.0194876, -0.0128811, -0.0133803, -0.0124343, -0.00238574, -9.01552e-05, -0.0158343, -0.0169781, -0.0258623, -0.0135792, -0.00127369, -0.000444025, -0.000290105, -0.0119873, -0.00249437, -0.000774275, -0.00205889, -0.0125666, -0.0229465, -0.0113158, -0.0269703, -0.0147568, -0.0165197, -0.023939, -0.0123759, -0.0410412, -0.00239118, -0.0015895, -0.000568142, -0.127523, -0.0288335, -0.0189847, -0.0121997, -0.000235778, -0.0011206, -0.0393002, -0.019633, -0.00279279, -0.0283847, -0.0453578, -0.0280013, -0.0601087, -0.0139329, -0.0600211, -0.0235795, -0.0356195, -0.7178, -0.0140968, -0.0030617, -0.0159083, -0.000131982, -0.0674715, -0.013471, -0.0123371, -0.0994951, -0.0191816, -0.00194142, -0.0158208, -0.000283217, -0.109854, -0.0202122, -0.0660837, -0.0475662, -0.0233339, -0.0464061, -0.000433421, -0.0103126, -0.0182202, -0.0160054, -0.000480109, -0.0165702, -0.0182133, -0.0169961, -0.0215591, -0.0148052, -0.0176536, -0.0207305, -0.0869287, -0.276504, -0.0122601, -0.286011, -0.225627, -0.340717, -0.0111441, -0.0181054, -0.0112728, -0.113165, -0.0221769, -0.0244429, -0.017351, -0.0143694, 9.24161e-06, -0.277328, -0.0214237, -0.0947139, 2.98961e-05, -0.010071, -0.032806, -0.0128048, -0.0206245, -0.0142076, -0.0101576, -0.0973298, -0.0125767, -0.0333357, -0.0208381, -0.0122846, -0.0261142, -0.146533, -0.0341015, -0.18178, -6.38799e-06, -0.0410061, -0.0164111, -0.0260626, -0.117376, -0.0610724, -0.0184515, -0.0132079, -0.0123247, -0.013155, -4.91189e-06, -0.0318437, -0.0297915, -0.0100724, -1.69551e-06, -0.0353501, -0.0148821, -0.0338786, -0.0316177, -0.0116572, -0.0393301, -0.173147, -0.017313, -0.0127027, -0.0172939, -0.0108093, -0.0148861, -0.0151126, -0.0189058, -0.0423173, 9.3261e-07, -0.0533387, -0.0135627, -0.011336, -0.229481, -0.0151969, -0.0126489, -0.0195398, -0.0120252, -0.0393834, 1.69437e-07, -0.0109137, 3.1643e-08, -0.0106776, -0.024027, -0.0118056, -0.0213015, -0.0219588, 1.97213e-08, 4.35993e-08, -0.0103069, -0.0126229, 1.36186e-07, -0.0145284, -0.0121896, -0.0401467, -0.0111547, -1.0353e-08, -8.46597e-09, -0.0110368, -0.139166, -0.0960186, -0.0137391, -0.0150605, -0.046281, -0.0237241, -0.01692, -0.0601792, -0.0254292, -0.0145106, -0.0912492, -0.011963, -0.0311922, -0.0123238, -0.0279163, -0.0202745, 6.92416e-09, -0.0193663, -0.0192289, 1.52601e-09, -0.0127945, -0.0118243, -0.0163521, -0.0160169, -0.0142504, -0.0129882, -0.013315, -0.0135864, -0.0152037, -0.0109274, -0.0874055, -6.1593e-10, -0.11194, -0.893142, -0.11194, -0.0874055, -0.0109274, -0.0152037, -0.0135864, -0.013315, -0.0129882, -0.0142505, -0.0160169, -0.0163521, -0.0118244, -0.0127945, -0.0192289, -0.0193663, -0.0202745, -0.0279163, -0.0123238, -0.0311922, -0.011963, -0.0912492, -0.0145105, -0.0254292, -0.0601792, -0.01692, -0.0237241, -0.0462809, -0.0150605, -0.0137388, -0.0960185, -0.139166, -0.0110368, -0.0111547, -0.0401467, -0.0121896, -0.0145285, -0.0126229, -0.0103069, -0.0219592, -0.021302, -0.0118063, -0.0240271, -0.010678, -0.0109141, -0.0393837, -0.0120256, -0.019541, -0.012649, -0.0151971, -0.229482, -0.0113381, -0.0135642, -0.0533413, -0.0423174, -0.0189059, -0.015113, -0.0148864, -0.0108097, -0.0172945, -0.0127028, -0.017313, -0.173147, -0.03933, -0.0116567, -0.0316172, -0.0338719, -0.0148804, -0.0353477, -0.0100683, -0.0297898, -0.0318411, -0.0131546, -0.0123189, -0.0132074, -0.0184488, -0.0610647, -0.117367, -0.0260586, -0.0164098, -0.0410046, -0.181772, -0.0341, -0.146533, -0.026116, -0.0122902, -0.0208433, -0.0333674, -0.0125843, -0.0973318, -0.0101801, -0.0142228, -0.0206866, -0.012819, -0.0328192, -0.0100734, -0.0947337, -0.021451, -0.277337, -0.0143934, -0.0173809, -0.02452, -0.0222867, -0.113294, -0.011295, -0.018146, -0.0111499, -0.340779, -0.225618, -0.285984, -0.0122254, -0.275372, -0.0861157, -0.0206271, -0.0171418, -0.0140279, -0.0208517, -0.01521, -0.0181478, -0.0158457, -0.0156625, -0.017999, -0.0101566, -0.0456084, -0.0228362, -0.0459569, -0.0650658, -0.0192101, -0.109316, -0.0150585, -0.017078, -0.0993982, -0.0113504, -0.0130421, -0.0673874, -0.0152459, -0.0140762, -0.71714, -0.0329289, -0.021309, -0.0600133, -0.0133072, -0.0534291, -0.0264861, -0.0437519, -0.0280156, -0.0178052, -0.0312603, -0.010855, -0.0136735, -0.0275592, -0.127032, -0.031339, -0.0111899, -0.0224905, -0.0151469, -0.012057, -0.0137199, -0.0111463, -0.0205015, -0.0117404, -0.011698, -0.0118644, -0.0233648, -0.016082, -0.0158329, -0.0123372, -0.0123351, -0.0124729, -0.0175809, -0.0122647, -0.016602, -0.0114351, -0.0104374, -0.0244485, -0.0249414, -0.0725154, -0.0187805, -0.156153, -0.0109741, -0.187516, -0.0112639, -0.0392307, -0.0119354, -0.0109741, -0.0546348, -0.0274259, -0.0251577, -0.0460874, -0.0110915, -0.0249501, -0.0251159, -0.0107088, -0.0107922, -0.028645, -0.014544, -0.168369, -0.0144967, -0.0761523, -0.0107264, -0.0291615, -0.0104749, -0.0121096, -0.0172494, -0.0290829, -0.0446091, -0.0115047, -0.0241055, -0.0295486, -0.0105011, -0.0737918, -0.017431, -0.0107865])

In [127]:
np.sum(e[e<0])

-18.7606225557249

In [129]:
np.sum([-0.0119001,
4.76118e-09,
-0.0109753,
1.73258e-08,
1.67668e-08,
-0.0249202,
6.74974e-09,
-0.0250545,
-0.0276426,
-0.0114597,
3.29748e-09,
-0.0309168,
2.42587e-09,
-0.0720774,
-0.985053,
-0.0158167,
-0.0192253,
-0.0186106,
9.64763e-09,
2.41803e-09,
1.148e-10,
1.90606e-09,
1.39773e-09,
-0.232461,
-0.913886,])

-2.3999991331888797